In [49]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier
import detoxify
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.preprocessing import OrdinalEncoder
import bertviz
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('D:/Stepik_datasets/Train_rev1.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   Title               244767 non-null  object
 2   FullDescription     244768 non-null  object
 3   LocationRaw         244768 non-null  object
 4   LocationNormalized  244768 non-null  object
 5   ContractType        65442 non-null   object
 6   ContractTime        180863 non-null  object
 7   Company             212338 non-null  object
 8   Category            244768 non-null  object
 9   SalaryRaw           244768 non-null  object
 10  SalaryNormalized    244768 non-null  int64 
 11  SourceName          244767 non-null  object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


## Data Preprocessing

Столбец `ContractType` содержит большое количество пропусков, поэтому удалим его полностью для дальнейшего анализа

In [4]:
display(df.loc[df.ContractTime == 'permanent'].head(20))
display(df.loc[df.ContractTime == 'contract'].head(20))

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
5,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk
6,14131336,Senior Subsea Pipeline Integrity Engineer,A globally renowned engineering and training c...,"Aberdeen, Borders",UK,NaN,permanent,Indigo 21 Ltd,Engineering Jobs,50000 - 100000/annum,75000,cv-library.co.uk
7,14663196,RECRUITMENT CONSULTANT INDUSTRIAL / COMMERCIA...,THIS IS A LIVE VACANCY NOT A GENERIC ADVERTISE...,"MANCHESTER, Greater Manchester",Manchester,NaN,permanent,Code Blue Recruitment,HR & Recruitment Jobs,18000 - 26000/annum TO 26K BASIC + COMM + BENS,22000,cv-library.co.uk
8,14663197,RECRUITMENT CONSULTANT CONSTRUCTION / TECHNIC...,This is an exceptional opportunity to join a c...,"LEEDS, West Yorkshire",Leeds,NaN,permanent,Code Blue Recruitment,HR & Recruitment Jobs,18000 - 28000/annum 18 - 28K BASIC + COMM + BENS,23000,cv-library.co.uk
9,15395797,Subsea Cables Engineer,A subsea engineering company is looking for an...,"Aberdeen, UK",Aberdeen,NaN,permanent,Indigo 21 Ltd,Engineering Jobs,70000 - 100000/annum,85000,cv-library.co.uk


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
39,28059808,Documentation Engineer,Our client is looking for a Documentation Engi...,Greater Manchester,Manchester,NaN,contract,Rullion Engineering Ltd,Engineering Jobs,25/hour,48000,cv-library.co.uk
42,28821429,Senior Planner,"Senior Planning Engineer, Nuclear Experience R...","Irlam, Seascale, Cumbria",UK,NaN,contract,Rullion Engineering Ltd,Engineering Jobs,35 - 45/hour,76800,cv-library.co.uk
46,29570885,Self Employed Sport Coaches,Do you have a passion for Sport? Can you motiv...,"Kirklees, West Yorkshire",UK,NaN,contract,Kirklees Active Leisure,Travel Jobs,8.84 to 12.91 per hour,20880,leisurejobs.com
54,30502270,Quality Engineer,Quality Engineer 6 month contract Location Fle...,"hampshire, South East, South East",Hampshire,NaN,contract,Gregory Martin International,IT Jobs,27000 - 30000/annum up to 30K,28500,cv-library.co.uk
92,34781787,Mechanical Fitter / Machine Tool Fitter,My Client who is based in Wolverhampton are lo...,"Wolverhampton, West Midlands",Wolverhampton,NaN,contract,Rullion Engineering Ltd,Manufacturing Jobs,14.39 - 17.55/hour Shift Allowance,30662,cv-library.co.uk
103,36757414,Primary Teachers,Are you a qualified/newly qualified teacher lo...,South Yorkshire,South Yorkshire,NaN,contract,Vision for Education,Teaching Jobs,90 - 140/day Excellent Rates of Pay,27600,cv-library.co.uk
153,43713923,Maintenance Electrician,"Our client, a global service provider, require...",London,London,NaN,contract,NaN,Creative & Design Jobs,12.50 - 13 per hour,24480,hays.co.uk
168,45358830,Waiter/Waitress,CHAMPNEYS HEALTH RESORTS JOB DESCRIPTION JOB T...,Tring,Tring,NaN,contract,Champneys Health Resort,Travel Jobs,5.25 - 6.11 p/h,10905,leisurejobs.com
1728,49715673,CNC Miller/Borers,We are currently looking for CNC Miller and Bo...,"Leeds, West Yorkshire, England, West Yorkshire",Leeds,NaN,contract,The Works Uk Ltd,Engineering Jobs,12/hour,23040,cv-library.co.uk
1742,49900634,Warehouse Administrator,We are keen to recruit a temporary warehouse a...,"Bradford, West Yorkshire, England, West Yorkshire",Bradford,NaN,contract,The Works Uk Ltd,Admin Jobs,6 - 7/hour,12480,cv-library.co.uk


In [5]:
df.drop('ContractType',axis=1,inplace=True)

df.Company.fillna('Other',inplace=True)

for i in ['Hour','hour','day','Day','week','Week','per','p/h']:
    df.loc[df.SalaryRaw.apply(lambda x: i in x),'ContractTime'] = 'contract'
    
df.loc[df.SalaryRaw.apply(lambda x: 'annum' in x),'ContractTime'] = 'permanent'
df.dropna(inplace=True)

Если сопоставить между собой два столбца : `ContractTime` и `SalaryRaw`, то можно заметить некую зависимость.

При `ContractTime` == `permanent` в столбце `SalaryRaw` чаще всего встречаются строки, содержащие конструкцию вида - "Зарплатная вилка/annum"

При `ContractTime` == `contract` в столбце `SalaryRaw` чаще всего встречаются строки, содержащие слова:
 - `Hour` \ `hour`
 - `Day` \ `day`
 - `Week` \ `Week`
 - `per`
 
Исходя из этого заполним пропуски в столбце `ContractTime` значениями `permanent`\ `contract` в зависимости от содержания столбца `SalaryRow`

In [6]:
# Немного сократим размер выборки, чтобы кол-во строк было кратно размеру батча(100) при создании эмбеддингов в дальнейшем
df = df[:225000]

In [7]:
# df = df.sample(20000).reset_index(drop='first')

Создадим сводную таблицу, где будет посчитано медианное значение зарплаты в зависимости от региона и категории

In [8]:
category_pivot = df.pivot_table(index=['Category', 'LocationNormalized'],
                                values='SalaryNormalized', aggfunc='median').reset_index()
category_pivot.columns = ['Category','LocationNormalized', 'category_median_salary']

Создадим новый признак в основном датафрейме путём приссоединения таблицы

In [9]:
df = df.merge(category_pivot,on = ['Category','LocationNormalized'])

In [10]:
df.head(10)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,category_median_salary
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,25000.0
1,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk,25000.0
2,65185048,Mechanical Design Engineer Vehicles,Mechanical Design Engineer Automotive Dorking...,Dorking Surrey South East,Dorking,permanent,Rise Technical Recruitment Ltd,Engineering Jobs,28000 - 36000 per annum + Bonus + Pension,32000,totaljobs.com,25000.0
3,69080369,Mechanical Design Engineer,Mechanical Design Engineer Dorking **** Ben...,"Dorking, Surrey",Dorking,permanent,Search Consultancy,Engineering Jobs,34000/annum Benefits,34000,cv-library.co.uk,25000.0
4,71552931,Engineering Consultant positions with World Cl...,Position Multiple Consultancy positions Salar...,Dorking,Dorking,permanent,JAM Recruitment,Engineering Jobs,6000 - 60000 per annum + Benefits,33000,MyUkJobs,25000.0
5,71821438,Multi Trade Fabric Engineer,Our client is a market leading facilities Mana...,"Dorking, South East",Dorking,permanent,Recru1tment Ltd,Engineering Jobs,27000/annum,27000,cv-library.co.uk,25000.0
6,72243818,Data Collectors,We are looking for people to work throughout t...,Dorking Surrey South East,Dorking,contract,CIP Recruitment Services Ltd,Engineering Jobs,6.50 per hour + performance-related bonus,12480,totaljobs.com,25000.0
7,72399868,Development Technician Electrical/Mechanical,Excellent role on offer for an expanding compa...,UK Dorking,Dorking,permanent,Rise Technical Recruitment Ltd,Engineering Jobs,22000 - 24000 per annum + Pension + Bonus,23000,careers4a.com,25000.0
8,72404880,Geotechnical / GeoEnvironmental Engineer,A Geotechnical / GeoEnivornmental Engineer is ...,"Dorking, Surrey",Dorking,permanent,Recruitment Genius,Engineering Jobs,21000 - 28000/annum,24500,cv-library.co.uk,25000.0
9,72441237,Assistant Production Manager,Main Purpose: To effectively manage daily acti...,Dorking Surrey South East,Dorking,permanent,Support Services Group,Engineering Jobs,"From 25,000 to 25,000 per annum",25000,totaljobs.com,25000.0


## BERT

In [11]:
# Определяем модель, токенизатор и веса "jobbert-base-cased"
model_class, tokenizer_class, pretrained_weights = (
    ppb.AutoModel, ppb.AutoTokenizer, "jjzha/jobbert-base-cased")

Создадим функцию, которая будет токенизировать переданные в неё столбцы и возвращать "padded-списки" и списки с "маской внимания"

In [12]:
%%time
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

def tokenizing(column_name):
    #Токенизируем наш текст и заменяем каждый токен на уникальый идентификатор 
    tokenized = df[column_name].apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True,max_length=512))

    #Дополняем каждую строку нулями до тех пор , пока не достигнем максимальной длины одной из строк
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)

    padded = np.array(
        [i + [0]*(max_len - len(i)) for i in tokenized.values])

    # Создаём маску "внимания", для того чтобы игнорировать добавленные нули
    attention_mask = np.where(padded != 0, 1, 0)
    return padded,attention_mask

CPU times: total: 62.5 ms
Wall time: 2.18 s


In [13]:
#Инициализируем модель
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at jjzha/jobbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at jjzha/jobbert-base-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.

Также создадим функцию, которая будет создавать эмбеддинги(768 шт.) для каждой конкретной фразы из столбца

In [15]:
%%time
#Подключаем видеокарту, Чтобы не умереть в ожидании))
device = torch.device("cuda")
model.cuda()

def make_emb(b_size,padded,attention_mask):
    batch_size = b_size
    embeddings = []
    for i in tqdm(range(padded.shape[0] // batch_size)):
            batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
            attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

            with torch.no_grad():
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
    return embeddings

CPU times: total: 1.25 s
Wall time: 183 ms


Токенизируем признаки `Title` и `FullDescription`, создадим эмбеддинги и приведем их к типу `DataFrame` для последующего объединения

In [16]:
title_padding,title_att_mask = tokenizing('Title')
title_emb = make_emb(1000,title_padding,title_att_mask)
title_features = np.concatenate(title_emb)
title_features = pd.DataFrame(title_features)

100%|██████████| 225/225 [05:00<00:00,  1.33s/it]


In [17]:
full_desc_padding,full_desc_att_mask = tokenizing('FullDescription')
full_desc_emb = make_emb(100,full_desc_padding,full_desc_att_mask)
full_desc_features = np.concatenate(full_desc_emb)
full_desc_features = pd.DataFrame(full_desc_features)

100%|██████████| 2250/2250 [52:43<00:00,  1.41s/it]


Изменим имена столбцов полученных эмбеддингов `Title` и `FullDescription`, т.к после преобразования эмбеддингов данные имена совпадают

In [18]:
title_col_name = [item+'t' for item in list(map(str, list(range(768))))]
full_desc_col_name = [item+'fd' for item in list(map(str, list(range(768))))]

title_features.columns = title_col_name
full_desc_features.columns = full_desc_col_name

## Feature Engineering

Соеденим эмбеддинги, полученные из столбцов `Title` и `FullDescription` в один датафрейм

In [19]:
features = title_features.join(full_desc_features)
target = df.SalaryNormalized

Закодируем признаки `Category`,`ContractTime`,`LocationNormalized` с помощью OHE

In [21]:
enc = OneHotEncoder(sparse=False,drop='first')
enc.fit(df[['Category','ContractTime','LocationNormalized']])
enc_tr = enc.transform(df[['Category','ContractTime','LocationNormalized']])

Закодируем признак `Company` с помощью OE 

In [23]:
ord_enc = OrdinalEncoder()
ord_enc.fit(df[['Company']])
ord_enc_tr = ord_enc.transform(df[['Company']])

Соеденим все "фичи" в один датафрейм и разделим его на тренировочную и тестовую выборку

In [24]:
features = features.join(pd.DataFrame(
    ord_enc_tr,columns=['Company']).set_index(features.index))

In [25]:
features = features.join(pd.DataFrame(
    enc_tr, columns=enc.get_feature_names()).set_index(features.index))

In [27]:
features = features.join([df.category_median_salary])

In [28]:
train_features,test_features, train_target, test_target = train_test_split(features,target,test_size=0.1,random_state=42)

In [29]:
train_target = np.log(train_target)

## Обучение

In [30]:
# time_list = []
# def teach_model(model, parameters, features, target,time_list,pipeline = False):
    
#     if pipeline == True:
#         imba_pipeline = make_pipeline(SMOTE(random_state=42,n_jobs=-1), 
#                                   model)   
#         grid = GridSearchCV(imba_pipeline, parameters, n_jobs=-1,
#                             scoring='f1',cv=3)
#     else:
#         grid = GridSearchCV(model, parameters, n_jobs=-1,
#                             scoring='neg_root_mean_squared_error',cv=3)
#     grid.fit(features, target)
#     print(grid.best_score_)
#     print(grid.best_params_)
    
#     time_list.append([grid.cv_results_['mean_fit_time'][0], grid.cv_results_[
#                      'mean_score_time'][0], grid.best_score_])

#     return grid.best_estimator_

In [37]:
%%time
# params = {'learning_rate':[.05,.01],de}
cb = CatBoostRegressor(random_seed=42,iterations=2000,learning_rate = .05)
cb.fit(train_features,train_target)
# best_cb = teach_model(model, params, train_features,train_target,time_list)


0:	learn: 0.4778295	total: 308ms	remaining: 10m 16s
1:	learn: 0.4679133	total: 607ms	remaining: 10m 6s
2:	learn: 0.4587011	total: 899ms	remaining: 9m 58s
3:	learn: 0.4501390	total: 1.18s	remaining: 9m 47s
4:	learn: 0.4421652	total: 1.45s	remaining: 9m 36s
5:	learn: 0.4347858	total: 1.73s	remaining: 9m 36s
6:	learn: 0.4278893	total: 2s	remaining: 9m 29s
7:	learn: 0.4214842	total: 2.26s	remaining: 9m 23s
8:	learn: 0.4156285	total: 2.55s	remaining: 9m 23s
9:	learn: 0.4101401	total: 2.81s	remaining: 9m 20s
10:	learn: 0.4050413	total: 3.08s	remaining: 9m 17s
11:	learn: 0.4002316	total: 3.36s	remaining: 9m 17s
12:	learn: 0.3958430	total: 3.63s	remaining: 9m 15s
13:	learn: 0.3914727	total: 3.9s	remaining: 9m 13s
14:	learn: 0.3876455	total: 4.17s	remaining: 9m 11s
15:	learn: 0.3839279	total: 4.44s	remaining: 9m 11s
16:	learn: 0.3805453	total: 4.72s	remaining: 9m 10s
17:	learn: 0.3773001	total: 4.98s	remaining: 9m 8s
18:	learn: 0.3742844	total: 5.24s	remaining: 9m 6s
19:	learn: 0.3714499	total:

In [46]:
r2_score(test_target,np.exp(cb.predict(test_features)))

0.6432627266816917

In [47]:
mean_squared_error(test_target,np.exp(cb.predict(test_features)))**.5

10362.316308126887

In [34]:
from sklearn.dummy import DummyRegressor

In [35]:
dummy = DummyRegressor()
dummy.fit(train_features,train_target)
dummy_predict = np.exp(dummy.predict(test_features))

print('r2:',r2_score(test_target,dummy_predict))
print('RMSE:',mean_squared_error(test_target,dummy_predict)**.5)

r2: -0.04837832669216202
RMSE: 17764.034899817227
